In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

<SparkContext master=yarn appName=PySparkShell>

In [2]:
! pip install pandas

In [4]:
! wget https://storage.googleapis.com/aekanunlab/LoanStats_web.csv

--2021-11-25 15:46:59--  https://storage.googleapis.com/aekanunlab/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.152.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... --2021-11-25 15:46:59--  https://storage.googleapis.com/aekanunlab/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.152.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1160157683 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv.3’

LoanStats_web.csv.3   0%[                    ]       0  --.-KB/s               200 OK
Length: 1160157683 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv.3’

LoanStats_web.csv.3 100%[===================>]   1.08G   136MB/s    in 5.7s    

2021-11-25

In [5]:
! hdfs dfs -mkdir -p /rawzone/

In [6]:
! hdfs dfs -put LoanStats_web.csv /rawzone/

put: `/rawzone/LoanStats_web.csv': File exists
put: `/rawzone/LoanStats_web.csv': File exists


# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [7]:
! ls -l ./LoanStats_web.csv

-rw-r--r-- 1 root root 1160157683 Sep  4  2019 ./LoanStats_web.csv
-rw-r--r-- 1 root root 1160157683 Sep  4  2019 ./LoanStats_web.csv


In [8]:
! head -3 ./LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [9]:
! wc -l ./LoanStats_web.csv

1432493 ./LoanStats_web.csv
1432493 ./LoanStats_web.csv


In [10]:
raw_df = spark.read.format('csv').option('header','true')\
.option('mode','DROPMALFORMED')\
.load('/rawzone/LoanStats_web.csv')

In [11]:
raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [12]:
len(raw_df.columns)

144

144

In [13]:
rawweb_df = raw_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [14]:
len(rawweb_df.columns)

73

73

In [15]:
rawweb_df.count()

1432466

1432466

In [16]:
selected_raw_df = rawweb_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [17]:
len(selected_raw_df.columns)

22

22

In [18]:
import pandas as pd
pd.DataFrame(selected_raw_df.describe().take(6), columns=selected_raw_df.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432440,15370.388358325654,9646.026272413836,1000,9975
term,1432440,None,None,36 months,60 months
int_rate,1432440,None,None,5.31%,30.99%
installment,1432440,456.68909888718275,281.7139997355062,100.01,999.98
grade,1432440,None,None,A,G
emp_length,1432440,None,None,1 year,n/a
home_ownership,1432440,None,None,ANY,RENT
annual_inc,1432440,81034.58293296752,134183.35696714383,0,99999.84
verification_status,1432440,None,None,Not Verified,Verified


,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432440,15370.388358325654,9646.026272413836,1000,9975
term,1432440,None,None,36 months,60 months
int_rate,1432440,None,None,5.31%,30.99%
installment,1432440,456.68909888718275,281.7139997355062,100.01,999.98
grade,1432440,None,None,A,G
emp_length,1432440,None,None,1 year,n/a
home_ownership,1432440,None,None,ANY,RENT
annual_inc,1432440,81034.58293296752,134183.35696714383,0,99999.84
verification_status,1432440,None,None,Not Verified,Verified


# 2. Data Preparation

## Data Cleansing: Remove missing values

In [19]:
no_missing_df = selected_raw_df.dropna(how='any')

## Check for a schema

In [20]:
no_missing_df.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nu

In [21]:
no_missing_df_fitmem = no_missing_df.repartition(60)

In [22]:
no_missing_df_cached = no_missing_df_fitmem.cache()

## Number of data rows

In [24]:
no_missing_df_cached.count()

1429645

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [25]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [26]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [27]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [28]:
from pyspark.sql.functions import col

In [29]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [30]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [31]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [32]:
crunched_df = no_missing_df_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [33]:
crunched_df.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

+---------+--------+-----+--------------+-----+----+
|loan_amnt|int_rate|  dti|home_ownership|grade|term|
+---------+--------+-----+--------------+-----+----+
|  10000.0|    5.32|14.98|      MORTGAGE|    A|36.0|
|  11000.0|   15.31|20.71|      MORTGAGE|    C|60.0|
|  11100.0|   15.31|32.51|      MORTGAGE|    C|36.0|
|   5325.0|   16.29|21.91|           OWN|    D|36.0|
|  12000.0|   15.31|18.88|          RENT|    C|60.0|
|  15400.0|   21.18|18.59|      MORTGAGE|    E|60.0|
|  21000.0|   14.46|13.87|      MORTGAGE|    C|60.0|
|   3000.0|   10.75|30.85|           OWN|    B|36.0|
|  20900.0|   18.99|23.52|          RENT|    D|36.0|
|  13500.0|    6.97| 21.1|      MORTGAGE|    A|36.0|
|  35000.0|   24.99|19.76|          RENT|    F|60.0|
|  12275.0|   14.46|20.17|          RENT|    C|36.0|
|  30000.0|   11.47|13.08|      MORTGAGE|    B|60.0|
|  35000.0|   12.99|24.36|      MORTGAGE|    C|36.0|
|  15000.0|   17.99|16.03|          RENT|    D|36.0|
|  18000.0|   18.99|28.44|          RENT|    D

In [34]:
len(crunched_df.columns)

22

22

In [35]:
crunched_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nu

In [36]:
crunched_df.count()

1429645

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [37]:
from pyspark.sql.functions import *

max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [38]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [39]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [40]:
max_loan_amnt = crunched_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = crunched_df.select(min('loan_amnt')).collect()[0][0]

In [41]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [42]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [43]:
normalized_df = crunched_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [44]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nu

### Number of data rows that are only "Fully Paid" and "Charged Off"

In [45]:
from pyspark.sql.functions import col

In [46]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|      0.0|36.0|    5.31|      30.12|    A|4.098360655737705E-4|   3215.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|5.081967213114754E-4|  25993.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|5.901639344262295E-4|   3311.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.065573770491804E-4|   1605.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1329.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1016.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|7.180327868852459E-4|    804.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|9.344262295081968E-4|    858.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|

In [47]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

### Drop Null

In [48]:
data_no_missing_df = normalized_filtered_df.dropna(how='any')

In [49]:
data_no_missing_df.count()

698373

698373

# 3. Data Modeling

In [50]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes\
, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [51]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [52]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [53]:
final_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [54]:
training , test = final_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

## Calculation for Confusion Matrix.

In [55]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [56]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 3]
    
    print (layers)
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [57]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [59]:
catcols = [\
           'home_ownership',\
           'grade',\
          ]

num_cols = [\
            'int_rate',\
            'installment',\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [60]:
from itertools import combinations, chain 
list(combinations(num_cols, 2))

[('int_rate', 'installment')]

[('int_rate', 'installment')]

In [61]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [62]:
result_list = []

In [ ]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        for k in range(len(num_cols)):
            for l in (list(combinations(num_cols, k+1))):
                l_list = list(l)
                if ('int_rate' in l_list) & ('installment' in l_list)\
                & ('home_ownership' in j) & ('grade' in j):
                    
                    ml_model = get_model(training,j,l_list,discols, splits, labelCol)
                    
                    lap2 = ml_model.transform(test)\
                    .withColumn('prediction_label',spark_labeling(col('prediction')))\
                    .toPandas()
                    
                    m2 = eval_metrics(lap2)
                    
                    result_text = "รอบแรกใช้ categorical {0} และ numerical {1} ได้ Model ที่มี {2}".format(j,l_list,m2)
                    
                    print(result_text)
                    
                    
                    result_list.append(result_text)
                

[15, 20, 10, 5, 3]
[15, 20, 10, 5, 3]
รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6068494009910306, 'Sensitivity': 0.6616289407098407, 'NPV': 0.6720029167080972, 'Specificity': 0.6179330102709457, 'Accuracy': 0.6385258713763153}
รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6068494009910306, 'Sensitivity': 0.6616289407098407, 'NPV': 0.6720029167080972, 'Specificity': 0.6179330102709457, 'Accuracy': 0.6385258713763153}


In [ ]:
result_list

["รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6068494009910306, 'Sensitivity': 0.6616289407098407, 'NPV': 0.6720029167080972, 'Specificity': 0.6179330102709457, 'Accuracy': 0.6385258713763153}"]

["รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6068494009910306, 'Sensitivity': 0.6616289407098407, 'NPV': 0.6720029167080972, 'Specificity': 0.6179330102709457, 'Accuracy': 0.6385258713763153}"]

In [ ]:
ml_model.transform(test)\
                    .withColumn('prediction_label',spark_labeling(col('prediction')))\
.groupBy('prediction_label','prediction','loan_status','indexedLabel').count().show()

+----------------+----------+-----------+------------+-----+
|prediction_label|prediction|loan_status|indexedLabel|count|
+----------------+----------+-----------+------------+-----+
|      Fully Paid|       0.0|Charged Off|         1.0| 9896|
|      Fully Paid|       0.0| Fully Paid|         0.0|20275|
|     Charged Off|       1.0|Charged Off|         1.0|19350|
|     Charged Off|       1.0| Fully Paid|         0.0|12536|
+----------------+----------+-----------+------------+-----+

+----------------+----------+-----------+------------+-----+
|prediction_label|prediction|loan_status|indexedLabel|count|
+----------------+----------+-----------+------------+-----+
|      Fully Paid|       0.0|Charged Off|         1.0| 9896|
|      Fully Paid|       0.0| Fully Paid|         0.0|20275|
|     Charged Off|       1.0|Charged Off|         1.0|19350|
|     Charged Off|       1.0| Fully Paid|         0.0|12536|
+----------------+----------+-----------+------------+-----+



In [ ]:
#ml_model.save('gs://aekanunlab/mlp_for_loanpayment-25nov2021/')